<a href="https://colab.research.google.com/github/aertist/ai/blob/main/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

!git clone https://github.com/dvschultz/stylegan2
!pip install opensimplex # needed for noise interpolation
%cd stylegan2
%mkdir datasets

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive/')

#Test the model
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

#Interpolation (Creating Videos)


In [ ]:
!python run_generator.py generate-latent-walk --network /content/drive/MyDrive/Models/art.pkl --seeds 11,17,25,34,78,90,21,45,67,87,44,65,85,12,32,40,11 --frames 500 --truncation-psi=0.5

In [ ]:
#convert to video 
!ffmpeg -r 25 -i ./results/00000-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p output.mp4

Let’s zip the generated files and download them.

In [ ]:
!zip -r interpolation_frames.zip /content/stylegan2/results/00000-generate-latent-walk 

#To simply generate images from the dataset


In [ ]:
!python run_generator.py generate-images --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3875451-3876000 --truncation-psi=0.7

In [ ]:
!zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

#Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). I recommend doing this on your server because the files become quite large and will be slow to upload over FTP.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [ ]:
python dataset_tool.py create_from_images ./datasets/my-custom-dataset ./my-custom-images

Once running, your training files will show up in the results folder.

#Training
I do not recommend attempting to train your model on Colab. It will be a very tedious process. Here are the steps, however.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.


In [ ]:
!python run_training.py --num-gpus=1 --data-dir=~/datasets --config=config-f --dataset=ffhq --mirror-augment=true